<a href="https://colab.research.google.com/github/friederrr/proof_contamination/blob/main/code/CDM_eval/CD_pipeline_CV_ngram_cdd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CD PIPELINE (mit CV) n-gram CDD

In [ ]:
%%capture
!pip install datasets

In [ ]:
#mount google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import random
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import torch
import json
from datetime import datetime
import itertools
from itertools import cycle, product
import re
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from pathlib import Path
import statistics
from sklearn.model_selection import StratifiedKFold

In [ ]:
from CDMs_functions_v1 import *

## Model Selektion

**OLMO**:

- "allenai/OLMo-7B-0724-hf": Downlaod (F32): (27 GB), GPU-RAM (bfloat16): 14 GB (T4 sufficient)

- "allenai/OLMo-7B-0724-SFT-hf": Download (BF16) (14 GB), GPU-RAM (14 GB)

- "allenai/OLMo-7B-0724-Instruct-hf": Download (BF16) (14 GB), GPU-RAM (14 GB)

- "allenai/OLMo-2-1124-13B-Instruct": Download (BF16) (28 GB), GPU-RAM (27 GB) -> works fine with A100!

- "allenai/OLMo-2-0325-32B-Instruct": Download (BF16) (65 GB), GPU-RAM (34 GB). It needs about 15min to answer one question with A100 (meta-device offloading). However, besides from that, the pipeline was compatible. Though, it is not recommended to use it with A100.

**DEEPSEEK**:
- "deepseek-ai/deepseek-math-7b-instruct":

**LEMMA**:
- "EleutherAI/llemma_7b"

In [ ]:
#!Parameter
model_id = "allenai/OLMo-2-1124-13B-Instruct"
DRIVE_PATH=Path('/content/drive/MyDrive/Masterarbeit25/')

In [ ]:
dir_token_path=DRIVE_PATH / "MathCONTA_ngram_cdd_generation_data"
file_paths=list_files_in_directory(dir_token_path, model_id)

Files in /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-2-1124-13B-Instruct:

OLMo-2-1124-13B-Instruct/notes.gdoc
OLMo-2-1124-13B-Instruct/MathCONTA_full_nstarts3_n10_20_30_sample50.json
OLMo-2-1124-13B-Instruct/MathCONTA_train_full_nstarts3_n10_20_30_sample50.json
OLMo-2-1124-13B-Instruct/MathCONTA_test_full_nstarts3_n10_20_30_sample50.json
Category_split/word_full_nstarts3_n10_20_30_sample50.json
Category_split/amc8_full_nstarts3_n10_20_30_sample50.json
Category_split/aime_full_nstarts3_n10_20_30_sample50.json
Category_split/Forum_full_nstarts3_n10_20_30_sample50.json
Category_split/MathCONTA_full_nstarts3_n10_20_30_sample50.json


In [ ]:
#merge_json_files(file_paths, dir_token_path / model_id / "MathCONTA_full_nstarts3_n10_20_30_sample50.json")

In [ ]:
#if file exists you don't have to load the models
load_models=False
if load_models:
  model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map='auto')
  tokenizer = AutoTokenizer.from_pretrained(model_id)
else:
  model = None
  tokenizer = AutoTokenizer.from_pretrained(model_id) #tokenizer needed for count
  ds_conta = None

## DATA Processing/Loading (Standard)

In [ ]:
#!Parameter
repo_id = "Tobstar001/MathCONTA"
split = "test"
config_name="core"

ds_conta = load_dataset(path=repo_id,name=config_name,split=split)
ds_conta_f = ds_conta.filter(lambda example: example["CATEGORY"] == "word-problems")

In [ ]:
#ds_conta_f = ds_conta.filter(lambda example: example["CATEGORY"] == "Forum")
model_config = {
    "model_id": model_id,
    "model": model,
    "tokenizer": tokenizer,
    "torch_seed": 42,
}

method_config = {
    "n_starts": 3,
    "sample_size": 50,
    "n_range": [10,20,30],
    "random_starts": False,
    "seed_rand": 42,
    "start_offset": 10,
    "verbose": False,

}

data_config = {
    "ds_conta": ds_conta,
    "only_problem": False,
    "force_reprocess": False,
    "data_name": "MathCONTA_full",
    "base_path_generation": DRIVE_PATH / "MathCONTA_ngram_cdd_generation_data"
}

ngram_cdd_data=create_ngram_cdd_generation_data(**model_config, **method_config, **data_config)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_full_nstarts3_n10_20_30_sample50.json


In [ ]:
ngram_cdd_df=get_ngram_cdd_df_from_generation_data(ngram_cdd_data,10,0.0)
ngram_cdd_df

,ID,CATEGORY,LABEL,LABEL_BINARY,n,alpha,ngram_cdd_mean
0,owm-amc8-1,AMC8,Conta,1,10,0.0,0.193333
1,owm-amc8-2,AMC8,Conta,1,10,0.0,0.120000
2,owm-amc8-3,AMC8,Conta,1,10,0.0,0.000000
3,owm-amc8-4,AMC8,Conta,1,10,0.0,0.100000
4,owm-amc8-5,AMC8,Conta,1,10,0.0,0.180000
...,...,...,...,...,...,...,...
95,clean-aime-6,AIME,Clean,0,10,0.0,0.000000
96,clean-aime-7,AIME,Clean,0,10,0.0,0.160000
97,clean-aime-8,AIME,Clean,0,10,0.0,0.333333
98,clean-aime-9,AIME,Clean,0,10,0.0,0.166667


## Train Test Split (Only relevant once)
Subsequent Stratified Train test split for google drive
For new data please filter train/test before generating the data

In [ ]:
train, test = stratified_dict_split(ngram_cdd_data, test_ratio=0.3, seed=42)

Train size: 70, Test size: 30
{'owm-amc8-9', 'owm-forum-3', 'owm-aime-3', 'clean-amc8-3', 'owm-amc8-12', 'clean-word-10', 'clean-amc8-14', 'clean-aime-7', 'clean-word-7', 'owm-word-9', 'clean-aime-3', 'owm-word-5', 'clean-forum-7', 'owm-word-10', 'clean-aime-4', 'owm-amc8-15', 'owm-aime-4', 'clean-word-12', 'clean-forum-6', 'owm-word-2', 'owm-amc8-10', 'clean-amc8-1', 'owm-forum-5', 'owm-aime-7', 'clean-amc8-18', 'owm-forum-10', 'clean-amc8-10', 'clean-word-9', 'owm-amc8-17', 'clean-forum-9'}


In [ ]:
#train_path: dir_token_path / model_id / "MathCONTA_train_sample50_max200_seed42.json", test path accordingly
file_path_train=dir_token_path / model_id / "MathCONTA_train_full_nstarts3_n10_20_30_sample50.json"
with open(file_path_train, 'w') as f:
      json.dump(train, f)

file_path_test=dir_token_path / model_id / "MathCONTA_test_full_nstarts3_n10_20_30_sample50.json"
with open(file_path_test, 'w') as f:
      json.dump(test, f)

## TRAIN SET CV
Find best parameters - CV

In [ ]:
model_config = {
    "model_id": model_id,
    "model": model,
    "tokenizer": tokenizer,
    "torch_seed": 42,
}

method_config = {
    "n_starts": 3,
    "sample_size": 50,
    "n_range": [10,20,30],
    "random_starts": False,
    "seed_rand": 42,
    "start_offset": 10,
    "verbose": False,

}

data_config = {
    "ds_conta": ds_conta,
    "only_problem": False,
    "force_reprocess": False,
    "data_name": "MathCONTA_train_full",
    "base_path_generation": DRIVE_PATH / "MathCONTA_ngram_cdd_generation_data"
}

ngram_cdd_data=create_ngram_cdd_generation_data(**model_config, **method_config, **data_config)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_train_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_train_full_nstarts3_n10_20_30_sample50.json


In [ ]:
data_dict = {
    "ngram_data": ngram_cdd_data,
    "feature_col": "ngram_cdd_mean",
    "label_col": "LABEL_BINARY",
    "data_name": "MathCONTA_train_full"
}

method_dict = {
    "n_range": [10, 20, 30],
    "alpha_range": [0.0,0.1,0.2,0.3],
    "cv_folds": 5,
    "cv_seed": 42,
    "method_name": "ngram_cdd"
}

model_dict = {
    "model_id": model_id,
    "log_path_base": DRIVE_PATH / "cdm_data" / "MathCONTA_v1",
    "exp_id": "overleaf2"#+"_"+category
}

# Call the function with the dictionaries
out_dict=tune_ngram_cdd(**data_dict, **method_dict, **model_dict)


Testing n=10,alpha0.0 | median_threshold=0.076667 | mean_cvacc_test=0.457143
Testing n=10,alpha0.1 | median_threshold=0.030000 | mean_cvacc_test=0.500000
Testing n=10,alpha0.2 | median_threshold=0.010000 | mean_cvacc_test=0.500000
Testing n=10,alpha0.3 | median_threshold=0.033333 | mean_cvacc_test=0.514286
Testing n=20,alpha0.0 | median_threshold=0.073333 | mean_cvacc_test=0.385714
Testing n=20,alpha0.1 | median_threshold=0.093333 | mean_cvacc_test=0.414286
Testing n=20,alpha0.2 | median_threshold=0.016667 | mean_cvacc_test=0.428571
Testing n=20,alpha0.3 | median_threshold=0.010000 | mean_cvacc_test=0.500000
Testing n=30,alpha0.0 | median_threshold=0.000000 | mean_cvacc_test=0.385714
Testing n=30,alpha0.1 | median_threshold=0.000000 | mean_cvacc_test=0.457143
Testing n=30,alpha0.2 | median_threshold=0.006667 | mean_cvacc_test=0.442857
Testing n=30,alpha0.3 | median_threshold=0.000000 | mean_cvacc_test=0.471429
Best log entry:
{'parameter': {'n': 10, 'alpha': 0.3}, 'global_threshold': 0

## TEST SET EVALUATION

In [ ]:
data_name="MathCONTA_test_full"
data_config.update({"data_name": data_name})
ngram_cdd_data=create_ngram_cdd_generation_data(**model_config, **method_config, **data_config)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_test_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-2-1124-13B-Instruct/MathCONTA_test_full_nstarts3_n10_20_30_sample50.json


In [ ]:
#!Parameter
parameter1=10
parameter2=0.3
theta=0.033333
data_name=data_name
exp_id_test=model_dict['exp_id']
log_path_base=model_dict['log_path_base']
model_id=model_dict['model_id']
method_name=method_dict['method_name']
metric_col=data_dict['feature_col']

ngram_cdd_df=get_ngram_cdd_df_from_generation_data(ngram_cdd_data,parameter1,parameter2)
len(ngram_cdd_df)

30

In [ ]:
test_dict=evaluate_method_standard(
    ngram_cdd_df,
    metric_col=metric_col,
    theta=theta,
    model_id=model_id,
    data_name=data_name,
    method_name=method_name,
    parameter1=parameter1,
    parameter2=parameter2,
    n_bootstrap=1000,
    seed=42
)
test_dict

{'model_id': 'allenai/OLMo-2-1124-13B-Instruct',
 'data_name': 'MathCONTA_test_full',
 'datetime': '2025-04-11T21:22:34.343427',
 'method_name': 'ngram_cdd',
 'parameter': {'parameter1': 10, 'parameter2': 0.3, 'theta': '0.033333'},
 'metrics': {'accuracy': 0.5,
  'accuracy_95CI': (0.3, 0.6666666666666666),
  'precision': 0.5,
  'recall': 1.0,
  'f1_score': 0.6666666666666666,
  'confusion_matrix': [[0, 15], [0, 15]],
  'mcnemar_b': 0,
  'mcnemar_c': 0,
  'mcnemar_p_value': 1.0},
 'n_bootstrap': 1000,
 'seed': 42}

In [ ]:
save_accuracy_log(log_path_base=log_path_base,model_id=model_id,method_name=method_name,exp_id=exp_id_test, data_name=data_name,out_dict=test_dict)

Accuracy log saved in /content/drive/MyDrive/Masterarbeit25/cdm_data/MathCONTA_v1/allenai/OLMo-2-1124-13B-Instruct/ngram_cdd/MathCONTA_test_full_accuracylog_overleaf2.json


## RUN END2END

In [ ]:
model_config = {
    "model_id": model_id,
    "model": model,
    "tokenizer": tokenizer,
    "torch_seed": 42,
}

method_config = {
    "n_starts": 3,
    "sample_size": 2,
    "n_range": [10,30],
    "random_starts": False,
    "seed_rand": 42,
    "start_offset": 10,
    "verbose": False,
    "n":10,
    "alpha": 0.2,
    "theta":0.125

}

data_config = {
    "ds_conta": None,
    "only_problem": False,
    "force_reprocess": False,
    "data_name": "MathCONTA_random5",
    "base_path_generation": DRIVE_PATH / "MathCONTA_ngram_cdd_generation_data"
}


In [ ]:
run_ngram_cdd(**model_config,**method_config,**data_config)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_random5_nstarts3_n10_30_sample2.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_random5_nstarts3_n10_30_sample2.json


{'accuracy': 1.0,
 'precision': 1.0,
 'recall': 1.0,
 'f1_score': 1.0,
 'confusion_matrix': array([[3, 0],
        [0, 2]])}

## Combined Evaluation across LLMs

!!Prerequisit: Train/Test splits already stored in DRIVE for the relevant models

In [ ]:
model_ids = ["deepseek-ai/deepseek-math-7b-instruct",
             "EleutherAI/llemma_7b",
             "allenai/OLMo-7B-0724-Instruct-hf",
              "allenai/OLMo-2-1124-13B-Instruct"]
model_id_combi = "DS_LE_OL_OL2"

### Combi CV train
Find best parameters - CV

In [ ]:
model_config = {
    "model_id": model_id,
    "model": model,
    "tokenizer": tokenizer,
    "torch_seed": 42,
}

method_config = {
    "n_starts": 3,
    "sample_size": 50,
    "n_range": [10,20,30],
    "random_starts": False,
    "seed_rand": 42,
    "start_offset": 10,
    "verbose": False,

}

data_config = {
    "ds_conta": ds_conta,
    "only_problem": False,
    "force_reprocess": False,
    "data_name": "MathCONTA_train_full",
    "base_path_generation": DRIVE_PATH / "MathCONTA_ngram_cdd_generation_data"
}

ngram_cdd_data_combined = []
for model_id in model_ids:
    model_config.update({"model_id": model_id})
    ngram_cdd_data=create_ngram_cdd_generation_data(**model_config, **method_config, **data_config)
    ngram_cdd_data_combined += ngram_cdd_data

len(ngram_cdd_data_combined)


File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_train_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_train_full_nstarts3_n10_20_30_sample50.json
File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/EleutherAI/llemma_7b/MathCONTA_train_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/EleutherAI/llemma_7b/MathCONTA_train_full_nstarts3_n10_20_30_sample50.json
File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-7B-0724-Instruct-hf/MathCONTA_train_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/

280

In [ ]:
data_dict = {
    "ngram_data": ngram_cdd_data_combined,
    "feature_col": "ngram_cdd_mean",
    "label_col": "LABEL_BINARY",
    "data_name": "MathCONTA_train_full"
}

method_dict = {
    "n_range": [10, 20, 30],
    "alpha_range": [0.0,0.1,0.2,0.3],
    "cv_folds": 5,
    "cv_seed": 42,
    "method_name": "ngram_cdd"
}

model_dict = {
    "model_id": model_id_combi,
    "log_path_base": DRIVE_PATH / "cdm_data" / "MathCONTA_v1",
    "exp_id": "overleaf2combi"#+"_"+category
}

# Call the function with the dictionaries
out_dict=tune_ngram_cdd(**data_dict, **method_dict, **model_dict)


Testing n=10,alpha0.0 | median_threshold=0.080000 | mean_cvacc_test=0.517857
Testing n=10,alpha0.1 | median_threshold=0.116667 | mean_cvacc_test=0.446429
Testing n=10,alpha0.2 | median_threshold=0.010000 | mean_cvacc_test=0.525000
Testing n=10,alpha0.3 | median_threshold=0.033333 | mean_cvacc_test=0.546429
Testing n=20,alpha0.0 | median_threshold=0.023333 | mean_cvacc_test=0.464286
Testing n=20,alpha0.1 | median_threshold=0.033333 | mean_cvacc_test=0.460714
Testing n=20,alpha0.2 | median_threshold=0.030000 | mean_cvacc_test=0.492857
Testing n=20,alpha0.3 | median_threshold=0.083333 | mean_cvacc_test=0.471429
Testing n=30,alpha0.0 | median_threshold=0.016667 | mean_cvacc_test=0.467857
Testing n=30,alpha0.1 | median_threshold=0.050000 | mean_cvacc_test=0.457143
Testing n=30,alpha0.2 | median_threshold=0.026667 | mean_cvacc_test=0.450000
Testing n=30,alpha0.3 | median_threshold=0.030000 | mean_cvacc_test=0.442857
Best log entry:
{'parameter': {'n': 10, 'alpha': 0.3}, 'global_threshold': 0

### Combi TEST SET EVALUATION

In [ ]:
data_name="MathCONTA_test_full"

ngram_cdd_data_combined = []
for model_id in model_ids:
    model_config.update({"model_id": model_id})
    data_config.update({"data_name": data_name})
    ngram_cdd_data=create_ngram_cdd_generation_data(**model_config, **method_config, **data_config)
    ngram_cdd_data_combined += ngram_cdd_data

len(ngram_cdd_data_combined)

File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_test_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/deepseek-ai/deepseek-math-7b-instruct/MathCONTA_test_full_nstarts3_n10_20_30_sample50.json
File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/EleutherAI/llemma_7b/MathCONTA_test_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/EleutherAI/llemma_7b/MathCONTA_test_full_nstarts3_n10_20_30_sample50.json
File path: /content/drive/MyDrive/Masterarbeit25/MathCONTA_ngram_cdd_generation_data/allenai/OLMo-7B-0724-Instruct-hf/MathCONTA_test_full_nstarts3_n10_20_30_sample50.json
Loading existing n-gram generation data from: /content/drive/MyDrive/Masterarbeit25/MathC

120

In [ ]:
#!Parameter
parameter1=10
parameter2=0.3
theta=0.033333
data_name=data_name
exp_id_test=model_dict['exp_id']
log_path_base=model_dict['log_path_base']
model_id=model_id_combi
method_name=method_dict['method_name']
metric_col=data_dict['feature_col']

ngram_cdd_df=get_ngram_cdd_df_from_generation_data(ngram_cdd_data_combined,parameter1,parameter2)
len(ngram_cdd_df)

120

In [ ]:
test_dict=evaluate_method_standard(
    ngram_cdd_df,
    metric_col=metric_col,
    theta=theta,
    model_id=model_id,
    data_name=data_name,
    method_name=method_name,
    parameter1=parameter1,
    parameter2=parameter2,
    n_bootstrap=1000,
    seed=42
)
test_dict

{'model_id': 'DS_LE_OL_OL2',
 'data_name': 'MathCONTA_test_full',
 'datetime': '2025-04-12T17:58:45.522756',
 'method_name': 'ngram_cdd',
 'parameter': {'parameter1': 10, 'parameter2': 0.3, 'theta': '0.033333'},
 'metrics': {'accuracy': 0.475,
  'accuracy_95CI': (0.38333333333333336, 0.5666666666666667),
  'precision': 0.48598130841121495,
  'recall': 0.8666666666666667,
  'f1_score': 0.6227544910179641,
  'confusion_matrix': [[5, 55], [8, 52]],
  'mcnemar_b': 5,
  'mcnemar_c': 8,
  'mcnemar_p_value': 0.5810546875},
 'n_bootstrap': 1000,
 'seed': 42}

In [ ]:
save_accuracy_log(log_path_base=log_path_base,model_id=model_id,method_name=method_name,exp_id=exp_id_test, data_name=data_name,out_dict=test_dict)

Accuracy log saved in /content/drive/MyDrive/Masterarbeit25/cdm_data/MathCONTA_v1/DS_LE_OL_OL2/ngram_cdd/MathCONTA_test_full_accuracylog_overleaf2combi.json
